<a href="https://colab.research.google.com/github/pmachadors/challengedados2/blob/main/semana3/semana3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pyspark
!wget "https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip" && unzip semanas-3-e-4.zip -d dados/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2022-12-11 16:51:37--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semanas-3-e-4.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 3.5.1.147, 52.217.101.236, 52.216.179.115, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|3.5.1.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588308 (2.5M) [application/zip]
Saving to: ‘semanas-3-e-4.zip.1’

semanas-3-e-4.zip.1 100%[===================>]   2.47M  2.24MB/s    in 1.1s    

2022-12-11 16:51:39 (2.24 MB/s) - ‘semanas-3-e-4.zip.1’ saved [2588308/2588308]

Archive:  semanas-3-e-4.zip
replace dados/dataset_ml_parquet/_SUCCESS? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dados/dataset_ml_parquet/._SUCCESS.crc? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Criando sessão Spark

In [51]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName('semana3').getOrCreate()

df_imoveis = spark.read.parquet('/content/dados/dataset_ml_parquet')

df_imoveis.show(5)


+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

# Clustering com VectorAssembler

In [52]:
from pyspark.ml.feature import VectorAssembler

x = ['andar',
    'area_util',
    'banheiros',
    'quartos',
    'suites',
    'vaga',
    'condominio',
    'iptu',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul',
    'Academia',
    'Animais permitidos',
    'Churrasqueira',
    'Condomínio fechado',
    'Elevador',
    'Piscina',
    'Playground',
    'Portaria 24h',
    'Portão eletrônico',
    'Salão de festas']

vect_features = VectorAssembler(inputCols=x, outputCol='features')
imoveis_vect = vect_features.transform(df_imoveis)

imoveis_vect.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|features                                                                                        |
+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+---

# Padronização dos dados

In [53]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
imoveis_scaled = scaler.fit(imoveis_vect).transform(imoveis_vect)

imoveis_scaled.show(5, truncate=False)

+------------------------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                                  |andar|area_util|banheiros|quartos|suites|vaga|bairro      |condominio|iptu |valor   |Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|

# Redução de Dimencionalidade com PCA

In [54]:
from pyspark.ml.feature import PCA

pca = PCA(k=5, inputCol='scaled_features',outputCol='pca_features')
model_pca = pca.fit(imoveis_scaled)
imoveis_pca = model_pca.transform(imoveis_scaled)

imoveis_pca.select('pca_features').show(5, truncate=False)


+------------------------------------------------------------------------------------------------------+
|pca_features                                                                                          |
+------------------------------------------------------------------------------------------------------+
|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954]   |
|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509]      |
|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536] |
|[-1.9528334788197674,2.054292371016602,-1.0129063602744157,-0.17794843497698848,-0.012607180989539779]|
|[-0.3550159242190174,2.168729788605275,0.5165722841101543,2.7636650294554017,-0.06840160430886463]    |
+------------------------------------------------------------------------------------------------------+
only showing top 5 rows



# Clustering com K-Means

In [63]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='pca_features', predictionCol='cluster_pca',k=5)
model_kmeans = kmeans.fit(imoveis_pca)

imoveis_kmeans = model_kmeans.transform(imoveis_pca)

imoveis_kmeans.select('id','pca_features','cluster_pca').show(5, truncate=False)

+------------------------------------+------------------------------------------------------------------------------------------------------+-----------+
|id                                  |pca_features                                                                                          |cluster_pca|
+------------------------------------+------------------------------------------------------------------------------------------------------+-----------+
|00002dd9-cc74-4809-b5a5-850adf0e7526|[-6.185066556186067,-1.5035861420376833,1.4021971813766747,-0.4627462052987944,0.05817557507559954]   |3          |
|0009ca94-2b37-4381-b8b8-773ce0f92444|[-3.2149125552925533,1.16996715944037,0.5164078437661357,3.2086930069839372,0.08994771500778509]      |4          |
|000e3d28-e3e5-4110-b488-69154931140e|[-1.0068958653993225,1.4962100951692308,2.6741317795672224,0.02967168175495706,-0.043207662071430536] |1          |
|000fb707-6cad-496d-8cb7-d8046cb5ef37|[-1.9528334788197674,2.054292371016602

# Filtrando imoveis

In [82]:
id = imoveis_kmeans.collect()[0]['id']
cluster = imoveis_kmeans.select('cluster_pca').where(f.col('id') == id).collect()[0][0]
imoveis_kmeans.where(f.col('cluster_pca') == cluster).show(5)



+--------------------+-----+---------+---------+-------+------+----+-----------------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+--------------------+--------------------+--------------------+-----------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|           bairro|condominio|  iptu|    valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|            features|     scaled_features|        pca_features|cluster_pca|
+--------------------+-----+---------+---------+-------+------+----+-----------------+----------+------+---------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+